In [8]:
import math
import numpy as np
from matplotlib import pyplot as plt
from PIL import Image
import cv2

threshold_values = {}
h = [1]


def Hist(img):
   row, col = img.shape 
   print(row,col)  #This comes out to be 311 X 569.
   y = np.zeros(256)
   for i in range(0,row):
      for j in range(0,col):
         y[img[i,j]] += 1
   x = np.arange(0,256)
   plt.bar(x, y, color='b', width=5, align='center', alpha=0.25)
   plt.show()
   return y


def regenerate_img(img, threshold):
    row, col = img.shape 
    y = np.zeros((row, col))
    for i in range(0,row):
        for j in range(0,col):
            if img[i,j] >= threshold:
                y[i,j] = 255
            else:
                y[i,j] = 0
    return y


   
def countPixel(h):
    cnt = 0
    for i in range(0, len(h)):
        if h[i]>0:
           cnt += h[i]
    return cnt


def wieght(s, e):
    w = 0
    for i in range(s, e):
        w += h[i]
    return w


def mean(s, e):
    m = 0
    w = wieght(s, e)
    for i in range(s, e):
        m += h[i] * i
    
    return m/float(w)


def variance(s, e):
    v = 0
    m = mean(s, e)
    w = wieght(s, e)
    for i in range(s, e):
        v += ((i - m) **2) * h[i]
    v /= w
    return v
            

def threshold(h):
    cnt = countPixel(h)
    for i in range(1, len(h)):
        vb = variance(0, i)
        wb = wieght(0, i) / float(cnt)
        mb = mean(0, i)
        
        vf = variance(i, len(h))
        wf = wieght(i, len(h)) / float(cnt)
        mf = mean(i, len(h))
        
        V2w = wb * (vb) + wf * (vf)
        V2b = wb * wf * (mb - mf)**2
        
        fw = open("trace.txt", "a")
        fw.write('T='+ str(i) + "\n")

        fw.write('Wb='+ str(wb) + "\n")
        fw.write('Mb='+ str(mb) + "\n")
        fw.write('Vb='+ str(vb) + "\n")
        
        fw.write('Wf='+ str(wf) + "\n")
        fw.write('Mf='+ str(mf) + "\n")
        fw.write('Vf='+ str(vf) + "\n")

        fw.write('within class variance='+ str(V2w) + "\n")
        fw.write('between class variance=' + str(V2b) + "\n")
        fw.write("\n")
        
        if not math.isnan(V2w):
            threshold_values[i] = V2w


def get_optimal_threshold():
    min_V2w = min(threshold_values.values())
    optimal_threshold = [k for k, v in threshold_values.items() if v == min_V2w]
    print('optimal threshold', optimal_threshold[0])
    return optimal_threshold[0]


image = Image.open('/content/img.jpg').convert("L")
img = np.asarray(image)

h = Hist(img)
threshold(h)
op_thres = get_optimal_threshold()

res = regenerate_img(img, op_thres)
plt.imshow(res)
plt.savefig("/content/otsu.jpg")


FileNotFoundError: [Errno 2] No such file or directory: '/content/img.jpg'

In [ ]:
import cv2 as cv
import matplotlib.pyplot as plt
import numpy as np
from google.colab.patches import cv2_imshow as im



# This is simply converting it to grayscale and applying the Otsu's thresholding.
img = cv.imread("/content/img.jpg",1)
gray = cv.cvtColor(img,cv.COLOR_BGR2GRAY)
th1 = 0
th2 = 255
ret,thresh = cv.threshold(gray,th1,th2,cv.THRESH_BINARY + cv.THRESH_OTSU)
   
    # noise removal
kernel = np.ones((3,3),np.uint8)
opening = cv.morphologyEx(thresh,cv.MORPH_OPEN,kernel, iterations = 2)
   
    # sure background area
sure_bg = cv.dilate(opening,kernel,iterations=3)
   
    # Defining accuracuy
acc = 0.01
   
    # Finding sure foreground area
dist_transform = cv.distanceTransform(opening,cv.DIST_L2,5)
ret, sure_fg = cv.threshold(dist_transform,acc*dist_transform.max(),255,0)
   
    # Finding unknown region
sure_fg = np.uint8(sure_fg)
unknown = cv.subtract(sure_bg,sure_fg)
   
    # Marker labelling
ret, markers = cv.connectedComponents(sure_fg)
 
    # Add one to all labels so that sure background is not 0, but 1
markers = markers+1
   
    # Now, mark the region of unknown with zero
markers[unknown==255] = 0
markers = cv.watershed(img,markers)
img[markers == -1] = [0,0,255]
   
   
cv.imwrite("content/segmentation.jpg",img)
im(img)
   
cv.waitKey(0)
cv.destroyAllWindows()

   

In [ ]:
#Before starting we should know what is countour exactly.
'''
Contours can be explained simply as a curve joining all the continuous points (along the boundary),
 having same color or intensity. 
 The contours are a useful tool for shape analysis and 
 object detection and recognition. 
 For better accuracy, use binary images.
'''

# import the necessary packages
from skimage.feature import peak_local_max
from skimage.morphology import watershed
from scipy import ndimage
import numpy as np
import argparse
import imutils
import cv2
from google.colab.patches import cv2_imshow as im

# construct the argument parse and parse the arguments
# ap = argparse.ArgumentParser()
# ap.add_argument("-i", "--image", required=True,
# 	help="path to input image")
# args = vars(ap.parse_args())

# # load the image and perform pyramid mean shift filtering
# # to aid the thresholding step
image = cv2.imread('/content/img.jpg')
shifted = cv2.pyrMeanShiftFiltering(image, 21, 51)
#cv2.imshow("Input", shifted)

# convert the mean shift image to grayscale, then apply
# Otsu's thresholding
gray = cv2.cvtColor(shifted, cv2.COLOR_BGR2GRAY)
thresh = cv2.threshold(gray, 0, 255,
	cv2.THRESH_BINARY | cv2.THRESH_OTSU)[1]
im(thresh)

# compute the exact Euclidean distance from every binary
# pixel to the nearest zero pixel, then find peaks in this
# distance map
D = ndimage.distance_transform_edt(thresh)
localMax = peak_local_max(D, indices=False, min_distance=20,
	labels=thresh)

# perform a connected component analysis on the local peaks,
# using 8-connectivity, then appy the Watershed algorithm with active countors.
markers = ndimage.label(localMax, structure=np.ones((3, 3)))[0]
labels = watershed(-D, markers, mask=thresh)
print("[INFO] {} unique segments found".format(len(np.unique(labels)) - 1))

# loop over the unique labels returned by the Watershed
# algorithm
for label in np.unique(labels):
	# if the label is zero, we are examining the 'background'
	# so simply ignore it
	if label == 0:
		continue

	# otherwise, allocate memory for the label region and draw
	# it on the mask
	mask = np.zeros(gray.shape, dtype="uint8")
	mask[labels == label] = 255

	# detect contours in the mask and grab the largest one
	cnts = cv2.findContours(mask.copy(), cv2.RETR_EXTERNAL,
		cv2.CHAIN_APPROX_SIMPLE)
	cnts = imutils.grab_contours(cnts)
	c = max(cnts, key=cv2.contourArea)

	# draw a circle enclosing the object
	((x, y), r) = cv2.minEnclosingCircle(c)
	cv2.circle(image, (int(x), int(y)), int(r), (0, 255, 0), 2)
	# cv2.putText(image, "#{}".format(label), (int(x) - 10, int(y)),
	# 	cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 0, 255), 2)

# show the output image
im(image)
cv2.imwrite("content/contour.jpg",image)  

In [1]:
import numpy as np
import cv2
from PIL import Image
k=1

# segmentation
seg = Image.open('../Result/Test/segmentation.jpg').convert("L")
seg = np.asarray(seg)
 
# ground truth
gt = Image.open('../Dataset/Test/gt.jpg').convert("L")
gt = np.asarray(seg)

dice = np.sum(seg[gt==k])*2.0 / (np.sum(seg) + np.sum(gt))

print("Dice similarity score is {}".format(dice))

Dice similarity score is 0.00019607138058098448


In [2]:
import numpy as np
import cv2
from PIL import Image
k=1

# segmentation
seg = Image.open('../Result/Test/contour.jpg').convert("L")
seg = np.asarray(seg)
 
# ground truth
gt = Image.open('../Dataset/Test/gt.jpg').convert("L")
gt = np.asarray(seg)

dice = np.sum(seg[gt==k])*2.0 / (np.sum(seg) + np.sum(gt))

print("Dice similarity score is {}".format(dice))

Dice similarity score is 0.0002464599353843849
